In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, roc_curve, auc
from sklearn.feature_selection import VarianceThreshold, SelectKBest, mutual_info_classif
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

# Set the number of cores for parallel processing
n_cores = max(1, os.cpu_count() // 2)
print(f"Using {n_cores} CPU cores for parallel processing")

# ====================== LOAD AND PREPROCESS DATA ======================
print("\n=== Loading and preprocessing data ===")
start_time = time.time()

# Load dataset
df = pd.read_csv("data/N300_G69_transposed.csv", dtype={'ssn':'Int64', 'type_of_attack': 'Int64', 'gen_attacked': 'Int64'})
print(f"Dataset loaded with shape: {df.shape}")

# Create binary target column if not already present
if 'attack' not in df.columns:
    df['attack'] = (df['type_of_attack'] != 0).astype(int)  # 1 for attack, 0 for no attack

# Display class distribution
attack_count = df['attack'].sum()
total_records = len(df)
normal_count = total_records - attack_count
print(f"Total records: {total_records}")
print(f"Number of attacks: {attack_count}")
print(f"Number of normal operations: {normal_count}")
print(f"Attack percentage: {attack_count/total_records:.2%}")

# Distribution of attack types
if 'type_of_attack' in df.columns:
    attack_types = df[df['attack'] == 1]['type_of_attack'].value_counts()
    print("\nAttack type distribution:")
    print(attack_types)
    
    # Map attack types to descriptive names
    attack_type_names = {
        0: "Normal",
        1: "Ramp Rate Attack",
        2: "Upper Limit Attack",
        3: "Lower Limit Attack",
        4: "Generation Cost Attack"
    }
    
    # Print distribution with names
    for attack_type, count in attack_types.items():
        print(f"{attack_type_names.get(attack_type, 'Unknown')}: {count} instances")

# Distribution of attacked generators
if 'gen_attacked' in df.columns:
    attacked_gens = df[df['attack'] == 1]['gen_attacked'].value_counts()
    print("\nTop 10 attacked generators:")
    print(attacked_gens.head(10))

# ====================== DOMAIN-SPECIFIC FEATURE ENGINEERING ======================
print("\n=== Performing domain-specific feature engineering ===")

# 1. Basic session and time features
df['day_in_ssn'] = df.groupby('ssn').cumcount() + 1  # Start at Day 1

# 2. Calculate OPF sensitivity features
# These help identify attacks on specific OPF parameters

# 2.1 Cost function sensitivity indicators
# Create features that might detect manipulation of generation costs (Type 4 attack)
df['fval_per_unit_load'] = df['fval'] / df.groupby('ssn')['fval'].transform('mean')
df['fval_normalized_by_ssn'] = df.groupby('ssn')['fval'].transform(
    lambda x: (x - x.mean()) / (x.std() if x.std() != 0 else 1))

# 2.2 Ramp rate indicators 
# Create features that might detect manipulation of ramp rates (Type 1 attack)
df['fval_change'] = df.groupby('ssn')['fval'].diff().fillna(0)
df['fval_change_rate'] = df['fval_change'] / df['fval'].shift(1).fillna(1)
df['fval_acceleration'] = df.groupby('ssn')['fval_change'].diff().fillna(0)

# Create rolling metrics to detect unusual ramp behavior
for window in [2, 3, 5]:
    # Rolling standard deviation of changes (volatility)
    df[f'fval_change_std_{window}d'] = df.groupby('ssn')['fval_change'].transform(
        lambda x: x.rolling(window=window, min_periods=1).std().fillna(0))
    
    # Maximum change in the window
    df[f'fval_max_change_{window}d'] = df.groupby('ssn')['fval_change'].transform(
        lambda x: x.rolling(window=window, min_periods=1).max().fillna(0))
    
    # Minimum change in the window
    df[f'fval_min_change_{window}d'] = df.groupby('ssn')['fval_change'].transform(
        lambda x: x.rolling(window=window, min_periods=1).min().fillna(0))

# 2.3 Generation limit indicators
# Create features that might detect manipulation of upper/lower limits (Type 2 & 3 attacks)
df['fval_peak_ratio'] = df['fval'] / df.groupby('ssn')['fval'].transform('max')
df['fval_trough_ratio'] = df['fval'] / df.groupby('ssn')['fval'].transform('min')

# Use quantiles to detect limits being approached
df['fval_quantile_in_ssn'] = df.groupby('ssn')['fval'].transform(
    lambda x: pd.qcut(x, q=10, labels=False, duplicates='drop').astype(float))

# 3. Create specialized detection features for each attack type
# These target the specific mechanisms of each attack type

# 3.1 Features for detecting ramp rate attacks (Type 1)
# Look for sudden discontinuities in how fast fval can change
df['ramp_constraint_active'] = (df['fval_change'].abs() > 
                               df.groupby('ssn')['fval_change'].transform('std')).astype(int)

# 3.2 Features for detecting generation limit attacks (Types 2 & 3)
# Look for values that should be infeasible under normal limits
df['upper_limit_proximity'] = 1 - (df['fval'] / df.groupby('ssn')['fval'].transform('max'))
df['lower_limit_proximity'] = (df['fval'] / df.groupby('ssn')['fval'].transform('min')) - 1

# 3.3 Features for detecting cost manipulation (Type 4)
# Look for cost-inefficient dispatches that shouldn't happen under normal cost functions
df['cost_efficiency'] = df['fval'] / df.groupby('ssn')['fval'].transform('mean')
df['cost_anomaly_score'] = df.groupby('ssn')['cost_efficiency'].transform(
    lambda x: (x - x.mean()).abs() / (x.std() if x.std() != 0 else 1))

# 4. Inter-session comparison features
# These help identify if a session is behaving differently from others

# 4.1 Calculate average fval across all sessions for each day
day_avg = df.groupby('day_in_ssn')['fval'].transform('mean')
day_std = df.groupby('day_in_ssn')['fval'].transform('std')

# 4.2 Compare each session's values to the average across all sessions
df['fval_day_deviation'] = (df['fval'] - day_avg) / (day_std if day_std.any() != 0 else 1)
df['fval_day_pct_diff'] = (df['fval'] - day_avg) / day_avg

# 5. Statistical anomaly detection features
# These help identify outliers regardless of attack mechanism

# 5.1 Z-scores and modified Z-scores for more robust outlier detection
df['fval_median_dev'] = df.groupby('ssn')['fval'].transform(
    lambda x: (x - x.median()) / (x.max() - x.min() if (x.max() - x.min()) != 0 else 1))

# 6. Cumulative features to detect subtle long-term manipulations
df['fval_cumsum'] = df.groupby('ssn')['fval'].cumsum()
df['fval_cummean'] = df.groupby('ssn')['fval'].expanding().mean().reset_index(level=0, drop=True)
df['fval_cum_deviation'] = df['fval'] - df['fval_cummean']

# 7. Trajectory features to detect changes in patterns
for lag in range(1, 4):
    df[f'fval_lag_{lag}'] = df.groupby('ssn')['fval'].shift(lag).fillna(0)
    df[f'fval_diff_lag_{lag}'] = df['fval'] - df[f'fval_lag_{lag}']



# Clean any NaN values
df = df.fillna(0)

# ====================== ADVANCED FEATURE ENGINEERING ======================
# Apply the advanced feature engineering 
from advanced_features import engineer_advanced_features
df = engineer_advanced_features(df)

# ====================== FEATURE SELECTION AND MODELING ======================
print("\n=== Preparing features for modeling ===")

# Define feature columns, excluding the target and direct identifiers
feature_columns = [col for col in df.columns if col not in ['attack', 'type_of_attack', 'gen_attacked']]

X = df[feature_columns]
y = df['attack']

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")

# Remove low variance features
print("\nApplying variance threshold...")
variance_threshold = 0.01
selector = VarianceThreshold(threshold=variance_threshold)
X_train_var = selector.fit_transform(X_train)
X_test_var = selector.transform(X_test)

print(f"Features after variance thresholding: {X_train_var.shape[1]}")

# Feature selection using mutual information
print("\nSelecting most informative features...")
select_k = SelectKBest(mutual_info_classif, k=min(100, X_train_var.shape[1]))
X_train_selected = select_k.fit_transform(X_train_var, y_train)
X_test_selected = select_k.transform(X_test_var)

print(f"Features after selection: {X_train_selected.shape[1]}")

# Print top 20 feature names
selected_indices = select_k.get_support(indices=True)
original_indices = selector.get_support(indices=True)
selected_names = [X.columns[original_indices[i]] for i in selected_indices[:20]]
scores = select_k.scores_[selected_indices]

print("\nTop 20 most informative features:")
for name, score in sorted(zip(selected_names, scores), key=lambda x: x[1], reverse=True)[:20]:
    print(f"{name}: {score:.6f}")

# Scale features
print("\nScaling features...")
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

# ====================== TRAIN MULTIPLE MODELS ======================
print("\n=== Training and evaluating models ===")

# Define a function to evaluate models
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Print results
    print(f"\n{model_name} Results:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("Confusion Matrix:")
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Normal', 'Attack'],
                yticklabels=['Normal', 'Attack'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f'Confusion Matrix - {model_name}')
    plt.savefig(f'{model_name.replace(" ", "_")}_confusion_matrix.png')
    plt.close()
    
    # ROC curve and AUC (if applicable)
    if hasattr(model, "predict_proba"):
        try:
            y_proba = model.predict_proba(X_test)[:, 1]
            fpr, tpr, _ = roc_curve(y_test, y_proba)
            roc_auc = auc(fpr, tpr)
            print(f"ROC AUC: {roc_auc:.4f}")
            
            # Plot ROC curve
            plt.figure()
            plt.plot(fpr, tpr, color='darkorange', lw=2, 
                    label=f'ROC curve (area = {roc_auc:.2f})')
            plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title(f'ROC Curve - {model_name}')
            plt.legend(loc="lower right")
            plt.savefig(f'{model_name.replace(" ", "_")}_roc_curve.png')
            plt.close()
        except Exception as e:
            print(f"Could not calculate ROC AUC: {e}")
            
    # Feature importance
    if hasattr(model, 'feature_importances_'):
        feature_names = [X.columns[original_indices[i]] for i in selected_indices]
        importances = model.feature_importances_
        indices = np.argsort(importances)[::-1]
        
        # Plot top 20 feature importances
        plt.figure(figsize=(12, 8))
        plt.title(f'Top 20 Feature Importances - {model_name}')
        plt.bar(range(min(20, len(importances))), 
                importances[indices[:20]], align='center')
        plt.xticks(range(min(20, len(importances))), 
                   [feature_names[i] for i in indices[:20]], rotation=90)
        plt.tight_layout()
        plt.savefig(f'{model_name.replace(" ", "_")}_feature_importance.png')
        plt.close()
        
        print("\nTop 10 most important features:")
        for i in range(min(10, len(importances))):
            print(f"{feature_names[indices[i]]}: {importances[indices[i]]:.6f}")
    
    return model, accuracy, f1

# ====================== MODEL 1: EXTRA TREES CLASSIFIER ======================
print("\n1. Training Extra Trees Classifier")

# Define parameter grid
et_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Create base model
et_base = ExtraTreesClassifier(random_state=42)

# RandomizedSearchCV
et_cv = RandomizedSearchCV(
    et_base, et_param_grid, n_iter=200, 
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring='accuracy', n_jobs=n_cores, random_state=42
)

# Fit on training data
et_cv.fit(X_train_scaled, y_train)

# Get best model
et_best = et_cv.best_estimator_
print(f"Best parameters: {et_cv.best_params_}")

# Evaluate
et_model, et_accuracy, et_f1 = evaluate_model(
    et_best, X_train_scaled, X_test_scaled, y_train, y_test, "Extra Trees"
)

# ====================== MODEL 2: GRADIENT BOOSTING ======================
print("\n2. Training Gradient Boosting Classifier")

# Define parameter grid
gb_param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'subsample': [0.8, 0.9, 1.0]
}

# Create base model
gb_base = GradientBoostingClassifier(random_state=42)

# RandomizedSearchCV
gb_cv = RandomizedSearchCV(
    gb_base, gb_param_grid, n_iter=200, 
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring='accuracy', n_jobs=n_cores, random_state=42
)

# Fit on training data
gb_cv.fit(X_train_scaled, y_train)

# Get best model
gb_best = gb_cv.best_estimator_
print(f"Best parameters: {gb_cv.best_params_}")

# Evaluate
gb_model, gb_accuracy, gb_f1 = evaluate_model(
    gb_best, X_train_scaled, X_test_scaled, y_train, y_test, "Gradient Boosting"
)

# ====================== MODEL 3: RANDOM FOREST ======================
print("\n3. Training Random Forest Classifier")

# Define parameter grid
rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Create base model
rf_base = RandomForestClassifier(random_state=42)

# RandomizedSearchCV
rf_cv = RandomizedSearchCV(
    rf_base, rf_param_grid, n_iter=200, 
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring='accuracy', n_jobs=n_cores, random_state=42
)

# Fit on training data
rf_cv.fit(X_train_scaled, y_train)

# Get best model
rf_best = rf_cv.best_estimator_
print(f"Best parameters: {rf_cv.best_params_}")

# Evaluate
rf_model, rf_accuracy, rf_f1 = evaluate_model(
    rf_best, X_train_scaled, X_test_scaled, y_train, y_test, "Random Forest"
)

# ====================== MODEL 4: XGBOOST ======================
print("\n4. Training XGBoost Classifier")

# Define parameter grid
xgb_param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.3],
    'max_depth': [3, 5, 7, 9],
    'min_child_weight': [1, 3, 5],
    'gamma': [0, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

# Create base model
xgb_base = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

# RandomizedSearchCV
xgb_cv = RandomizedSearchCV(
    xgb_base, xgb_param_grid, n_iter=200, 
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring='accuracy', n_jobs=n_cores, random_state=42
)

# Fit on training data
xgb_cv.fit(X_train_scaled, y_train)

# Get best model
xgb_best = xgb_cv.best_estimator_
print(f"Best parameters: {xgb_cv.best_params_}")

# Evaluate
xgb_model, xgb_accuracy, xgb_f1 = evaluate_model(
    xgb_best, X_train_scaled, X_test_scaled, y_train, y_test, "XGBoost"
)

# ====================== MODEL 5: LOGISTIC REGRESSION ======================
print("\n5. Training Logistic Regression")

# Define parameter grid
lr_param_grid = {
    'C': np.logspace(-3, 3, 7),
    'penalty': ['l1', 'l2', None],
    'solver': ['liblinear', 'saga', 'lbfgs'],
    'max_iter': [1000, 2000]
}

# Create base model
lr_base = LogisticRegression(random_state=42)

# RandomizedSearchCV
lr_cv = RandomizedSearchCV(
    lr_base, lr_param_grid, n_iter=50, 
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring='accuracy', n_jobs=n_cores, random_state=42
)

# Fit on training data
lr_cv.fit(X_train_scaled, y_train)

# Get best model
lr_best = lr_cv.best_estimator_
print(f"Best parameters: {lr_cv.best_params_}")

# Evaluate
lr_model, lr_accuracy, lr_f1 = evaluate_model(
    lr_best, X_train_scaled, X_test_scaled, y_train, y_test, "Logistic Regression"
)

# ====================== ENSEMBLE MODEL: VOTING CLASSIFIER ======================
print("\n6. Training Voting Classifier Ensemble")

# Create a dictionary of our models
models = {
    'ExtraTrees': et_best,
    'GradientBoosting': gb_best,
    'RandomForest': rf_best,
    'XGBoost': xgb_best,
    'LogisticRegression': lr_best
}

# Calculate scores
model_scores = {
    'ExtraTrees': et_f1,
    'GradientBoosting': gb_f1,
    'RandomForest': rf_f1,
    'XGBoost': xgb_f1,
    'LogisticRegression': lr_f1
}

# Sort by F1 score and select top 3
top_models = sorted(model_scores.items(), key=lambda x: x[1], reverse=True)[:3]
print(f"Top 3 models for ensemble: {[model[0] for model in top_models]}")

# Create voting classifier with top 3 models
estimators = [(name, models[name]) for name, _ in top_models]
voting_clf = VotingClassifier(estimators=estimators, voting='soft')

# Evaluate
voting_model, voting_accuracy, voting_f1 = evaluate_model(
    voting_clf, X_train_scaled, X_test_scaled, y_train, y_test, "Voting Classifier"
)

# ====================== SUMMARY ======================
print("\n=== Model Performance Summary ===")
models_summary = {
    'Extra Trees': (et_accuracy, et_f1),
    'Gradient Boosting': (gb_accuracy, gb_f1),
    'Random Forest': (rf_accuracy, rf_f1),
    'XGBoost': (xgb_accuracy, xgb_f1),
    'Logistic Regression': (lr_accuracy, lr_f1),
    'Voting Classifier': (voting_accuracy, voting_f1)
}

# Sort by F1 score
sorted_models = sorted(models_summary.items(), key=lambda x: x[1][1], reverse=True)

print("\nModels ranked by F1 score:")
for i, (model_name, (acc, f1)) in enumerate(sorted_models, 1):
    print(f"{i}. {model_name}: Accuracy = {acc:.4f}, F1 = {f1:.4f}")

# Save the best model
best_model_name = sorted_models[0][0]
best_model = None

if best_model_name == 'Extra Trees':
    best_model = et_model
elif best_model_name == 'Gradient Boosting':
    best_model = gb_model
elif best_model_name == 'Random Forest':
    best_model = rf_model
elif best_model_name == 'XGBoost':
    best_model = xgb_model
elif best_model_name == 'Logistic Regression':
    best_model = lr_model
else:
    best_model = voting_model

import joblib
joblib.dump(best_model, f'best_model_{best_model_name.replace(" ", "_")}.pkl')
print(f"\nBest model ({best_model_name}) saved as 'best_model_{best_model_name.replace(' ', '_')}.pkl'")



Using 2 CPU cores for parallel processing

=== Loading and preprocessing data ===
Dataset loaded with shape: (1920, 34036)
Total records: 1920
Number of attacks: 960
Number of normal operations: 960
Attack percentage: 50.00%

Attack type distribution:
type_of_attack
3    276
1    240
2    236
4    208
Name: count, dtype: Int64
Lower Limit Attack: 276 instances
Ramp Rate Attack: 240 instances
Upper Limit Attack: 236 instances
Generation Cost Attack: 208 instances

Top 10 attacked generators:
gen_attacked
25    40
60    36
19    28
39    28
52    28
12    28
29    28
61    28
23    28
16    24
Name: count, dtype: Int64

=== Performing domain-specific feature engineering ===

=== Performing advanced feature engineering ===

1. Creating specialized Lower Limit Attack detection features...
3. Creating features for large changes without ramp constraint activation...
4. Creating temporal pattern detection features...
5. Creating composite attack-specific features...
6. Creating interaction fe

In [2]:
import os
import glob
import shutil

def organize_output_files():
    """
    Organize output files from the power grid attack detection model:
    - Move all PNG files to a 'dfigures' folder
    - Move all model files (.pkl) to a 'models' folder
    """
    # Create directories if they don't exist
    os.makedirs('dfigures', exist_ok=True)
    os.makedirs('models', exist_ok=True)
    
    print("Organizing output files...")
    
    # Move PNG files to 'dfigures' folder
    png_files = glob.glob('*.png')
    for file in png_files:
        dest_path = os.path.join('dfigures', file)
        if os.path.exists(dest_path):
            os.remove(dest_path)  # Remove if exists to avoid errors
        shutil.move(file, 'dfigures')
        print(f"Moved {file} to dfigures/")
    
    # Move model files to 'models' folder
    model_files = glob.glob('*.pkl')
    for file in model_files:
        dest_path = os.path.join('models', file)
        if os.path.exists(dest_path):
            os.remove(dest_path)  # Remove if exists to avoid errors
        shutil.move(file, 'models')
        print(f"Moved {file} to models/")
    
    print(f"Organization complete. {len(png_files)} figures moved to 'dfigures' folder and {len(model_files)} models moved to 'models' folder.")

if __name__ == "__main__":
    organize_output_files()

Organizing output files...
Moved Extra_Trees_confusion_matrix.png to dfigures/
Moved Extra_Trees_feature_importance.png to dfigures/
Moved Extra_Trees_roc_curve.png to dfigures/
Moved Gradient_Boosting_confusion_matrix.png to dfigures/
Moved Gradient_Boosting_feature_importance.png to dfigures/
Moved Gradient_Boosting_roc_curve.png to dfigures/
Moved Logistic_Regression_confusion_matrix.png to dfigures/
Moved Logistic_Regression_roc_curve.png to dfigures/
Moved Random_Forest_confusion_matrix.png to dfigures/
Moved Random_Forest_feature_importance.png to dfigures/
Moved Random_Forest_roc_curve.png to dfigures/
Moved Voting_Classifier_confusion_matrix.png to dfigures/
Moved Voting_Classifier_roc_curve.png to dfigures/
Moved XGBoost_confusion_matrix.png to dfigures/
Moved XGBoost_feature_importance.png to dfigures/
Moved XGBoost_roc_curve.png to dfigures/
Moved best_model_Gradient_Boosting.pkl to models/
Organization complete. 16 figures moved to 'dfigures' folder and 1 models moved to 'm

In [3]:
# Add this to the end of your existing notebook

print("\n=== Analyzing Missed Attacks ===")

# Get the best model predictions on the test set
y_pred = best_model.predict(X_test_scaled)

# Create a DataFrame with test set results
test_results = pd.DataFrame({
    'actual': y_test,
    'predicted': y_pred,
    'correct': y_test == y_pred
})

# Get the original indices from the test set
test_indices = X_test.index.tolist()
test_results['original_index'] = test_indices

# Add the test results back to the original DataFrame for analysis
df_test_subset = df.loc[test_indices].copy()
df_test_subset['predicted'] = y_pred

# Find the missed attacks (false negatives)
missed_attacks = df_test_subset[(df_test_subset['attack'] == 1) & 
                               (df_test_subset['predicted'] == 0)]

# Display information about missed attacks
print(f"\nTotal missed attacks: {len(missed_attacks)}")
print("\nDistribution of missed attack types:")
missed_types = missed_attacks['type_of_attack'].value_counts()
for attack_type, count in missed_types.items():
    print(f"  Attack Type {attack_type} ({attack_type_names.get(attack_type, 'Unknown')}): {count} instances")

print("\nDistribution of missed generators:")
missed_gens = missed_attacks['gen_attacked'].value_counts().head(10)
print(missed_gens)

# Create a detailed report of all missed attacks
missed_attacks_report = missed_attacks[['original_index', 'type_of_attack', 'gen_attacked']].copy()
missed_attacks_report['attack_name'] = missed_attacks_report['type_of_attack'].map(attack_type_names)
missed_attacks_report = missed_attacks_report.sort_values('type_of_attack')

print("\nDetailed report of missed attacks (first 20 rows):")
print(missed_attacks_report.head(20))

# Save the full report to CSV
missed_attacks_report.to_csv('missed_attacks_report.csv', index=False)
print(f"\nFull report saved to 'missed_attacks_report.csv' with {len(missed_attacks_report)} rows")

# Optional: Analyze if missed attacks have patterns
print("\nAnalyzing patterns in missed attacks...")

# Compare feature distributions between correctly detected and missed attacks
attack_data = df_test_subset[df_test_subset['attack'] == 1].copy()
attack_data['detected'] = attack_data['predicted'] == 1

# Analyze a few key features
important_features = [
    'fval_change', 
    'fval_change_rate',
    'fval_diff_lag_1',
    'ramp_constraint_active',
    'fval_change_std_3d',
    'upper_limit_proximity',
    'lower_limit_proximity',
    'cost_anomaly_score'
]

for feature in important_features:
    if feature in attack_data.columns:
        detected_mean = attack_data[attack_data['detected']].get(feature, pd.Series()).mean()
        missed_mean = attack_data[~attack_data['detected']].get(feature, pd.Series()).mean()
        print(f"{feature}: Detected mean = {detected_mean:.4f}, Missed mean = {missed_mean:.4f}")

# Analyze if certain attack types are more likely to be missed
detected_by_type = attack_data.groupby('type_of_attack')['detected'].mean()
print("\nDetection rate by attack type:")
for attack_type, detection_rate in detected_by_type.items():
    print(f"  {attack_type_names.get(attack_type, 'Unknown')}: {detection_rate:.2%} detected")

# Analyze if certain generators are more likely to have missed attacks
detected_by_gen = attack_data.groupby('gen_attacked')['detected'].mean()
print("\nGenerators with lowest detection rates (top 5):")
print(detected_by_gen.sort_values().head(5))


=== Analyzing Missed Attacks ===

Total missed attacks: 0

Distribution of missed attack types:

Distribution of missed generators:
Series([], Name: count, dtype: Int64)


KeyError: "['original_index'] not in index"